In [1]:
import cv2
import numpy as np
import codecs
import math
from cv2 import line,resize, imwrite, imshow, boundingRect, countNonZero, cvtColor, drawContours, findContours, getStructuringElement, imread, morphologyEx, pyrDown, rectangle, threshold

In [2]:
def text_extraction(rgb,n):
    hasText=0
    height = np.size(rgb, 0)
    width = np.size(rgb, 1)
    ratio = height/width
    if ratio>1.2:
        rgb = resize(rgb,(800,1100))
    elif ratio<1.2 and ratio >1.0:
        rgb = resize(rgb,(600,500))
    else:
        rgb = resize(rgb,(700,400))
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY);
    morphKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,3))
    grad = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, morphKernel)
    _, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
    connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, morphKernel)
    mask = np.zeros(bw.shape[:2], dtype="uint8");
    _,contours, hierarchy = cv2.findContours(connected, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    j=0
    idx = 0
    while idx >= 0 :
        x,y,w,h = cv2.boundingRect(contours[idx]);
        cv2.drawContours(mask, contours, idx, (255, 255, 255), cv2.FILLED);     
        r = cv2.contourArea(contours[idx])/(w*h)
        if(r > 0.15 and h > 10 and h < 50 and w>20 and w < 500):
            cv2.rectangle(rgb, (x,y), (x+w,y+h), (0, 0, 0), 1)
            roi = rgb[y:y+h, x:x+w]
            imwrite("roi"+str(idx)+".jpg", roi)
            j = j+1
            hasText = 1
        idx = hierarchy[0][idx][0]
    imwrite("image"+ str(n)+".jpg", rgb)

In [3]:
def background_removal(dst,x):
    imga = cv2.Canny(dst,100,200)
    kernel = np.ones((3,3),np.uint8)
    imga = cv2.dilate(imga,kernel,iterations = 1)
    height = np.size(imga, 0)
    width = np.size(imga, 1)
    i = 0
    while(i<height):
        count = 0
        for j in range(0,width):
            if imga[i,j] == 255:
                count=count+1
        i = i+1
        if count >= width/10:
            break
    x1 = i

    i = height-1
    while(i>=0):

        count = 0
        for j in range(0,width):
            if imga[i,j] == 255:
                count=count+1
        i = i-1
        if count >= width/10:
            break
    x2 = i

    i = 0
    while(i<width):
        count = 0
        for j in range(0,height):
            if imga[j,i] == 255:
                count=count+1
        i = i+1
        if count >= height/10: 
            break
    y1 = i

    i = width-1
    while(i>=0):
        count = 0
        j = 0
        while(j<height):
            if imga[j,i] == 255:
                count=count+1
            j = j+1
        i = i-1
        if count >= height/10:
            break
    y2 = i
    crop_img = dst[x1-5:x2+5, y1-5:y2+5]
    imwrite(r"C:\Users\u293217\Downloads\\x"+ str(x)+".jpg", crop_img)
    text_extraction(crop_img,x)

In [4]:
def deskew(img,x):
    imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img1 = cv2.Canny(imgray,100,200)
    img2 = cv2.GaussianBlur(img1,(3,3),0)
    ret,thresh = cv2.threshold(img2,127,255,0)
    im2, contours, hierarchy = cv2.findContours(img2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    i=0
    max = 0
    while i<len(contours):
        area = cv2.contourArea(contours[i])
        if area>=max:
            max = area
            n = i;
        i=i+1  
    cv2.drawContours(img, contours, n, (0,255,0), 1)
    edges = cv2.Canny(img,50,150,apertureSize = 3)
    lines = cv2.HoughLines(edges,1,np.pi/180,50)
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),1)
    if x2-x1==0:
        angle = 90
    else:
        m = (y2-y1)/(x2-x1)
        angle = math.atan(m)
    angle = angle*180*7/22
    rows = np.size(img, 0)
    cols = np.size(img, 1)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),360+angle,1)
    image1 = imread(str(x) + ".jpg")
    dst = cv2.warpAffine(image1,M,(cols,rows))
    background_removal(dst,x)

In [5]:
def check_skew(rgb,n):
    image = rgb
    hasText=0
    height = np.size(rgb, 0)
    width = np.size(rgb, 1)
    ratio = height/width
    if ratio>1.2:
        rgb = resize(rgb,(800,1100))
    elif ratio<1.2 and ratio >1.0:
        rgb = resize(rgb,(600,500))
    else:
        rgb = resize(rgb,(700,400))
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY);
    morphKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,3))
    grad = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, morphKernel)
    _, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    morphKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
    connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, morphKernel)
    mask = np.zeros(bw.shape[:2], dtype="uint8");
    _,contours, hierarchy = cv2.findContours(connected, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    j=0
    idx = 0
    while idx >= 0 :
        x,y,w,h = cv2.boundingRect(contours[idx]);
        r = cv2.contourArea(contours[idx])/(w*h)
        if(r > 0.5 and h > 15 and w > 10 and h < 60):
            roi = rgb[y:y+h, x:x+w]
            j = j+1
            hasText = 1
        idx = hierarchy[0][idx][0]
    if j>5:
        background_removal(image,n)
    else:
        deskew(image,n)

In [7]:
for i in range(11,12,1):
        img = imread(r"C:\Users\u293217\Downloads\\" + str(i) + ".jpg")
        check_skew(img,i)